In [15]:
import numpy as np
from numpy import array
# import matplotlib.pyplot as plt
import time
from itertools import combinations

cumulative_percentages = array([0.000154,0.0015,0.0256,0.17,0.367,0.76,2.87,7.62,49.9][::-1])/100
names = ["Nothing", "Pair", "Two pair", "Three of a kind", "Straight", "Flush", "Full house", "Four of a kind", "Straight flush", "Royal flush"]


In [141]:
def get_shuffled_deck():
    arr = array([array([np.linspace(0,12,13).astype(int),(np.ones(13)*i).astype(int)]).T for i in range(4)]).reshape(52,2)
    
    np.random.shuffle(arr)
    return arr

def check_for_pairs(cards):
    _, uniques = np.unique(cards[:,0],return_counts=True)
    if 4 in uniques:
        return "FourOfAKind"
    elif 3 in uniques:
        if 2 in uniques:
            return "FullHouse"
        return "ThreeOfAKind"
    elif 2 in uniques:
        if sum(uniques == 2)==2:
            return "TwoPairs"
        return "Pair"
    return None

def check_for_straights(cards):
    sort = np.sort(cards[:,0])
    if sum((sort[1:] - sort[:-1]) != 1) == 0:
        if sort[-1] == 12:
            return "RoyalStraight"
        return "Straight"
    return None

def check_for_flush(cards):
    _, unique_suits = np.unique(cards[:,1], return_counts = True)

    if 5 in unique_suits:
        return "Flush"
    return None

def get_hand_value(cards):
    if cards.shape[0] != 5:
        print("Total cards are",cards.shape[0])
        return

    cv = get_combination_value(cards)

    high = np.max(cards[:,0])

    return 13*cv + high

def get_combination_value(cards):
    """
    0 Nothing
    1 Pair
    2 Two pair
    3 Three of a kind
    4 Straight
    5 Flush
    6 Full house
    7 Four of a kind
    8 Straight flush
    9 Royal flush
    """

    pairs = check_for_pairs(cards)
    straight = check_for_straights(cards)
    flush = check_for_flush(cards)
    
    
    if flush == "Flush" and straight == "RoyalStraight":
        return 9
    if flush == "Flush" and straight == "Straight":
        return 8
    if pairs == "FourOfAKind":
        return 7
    if pairs == "FullHouse":
        return 6
    if flush == "Flush":
        return 5
    if straight == "Straight":
        return 4
    if pairs == "ThreeOfAKind":
        return 3
    if pairs == "TwoPairs":
        return 2
    if pairs == "Pair":
        return 1
    return 0
   

class Game():
    def __init__(self):
        self.hand = array([])
        self.opponent_hand = array([])
        self.open_cards = array([])

    def start_game(self):
        self.deck = get_shuffled_deck()

    def draw_hand(self):
        self.hand = self.deck[:2]
        self.deck = self.deck[2:]

    def get_hand_value(self,hand):
        return get_hand_value(np.concatenate((self.open_cards,hand)))

    def draw_opponent_hand(self):
        self.opponent_hand = self.deck[:2]
        self.deck = self.deck[2:]
    
    def show_card(self):
        self.open_cards = np.append(self.open_cards, self.deck[0]).reshape(-1,2).astype(int)
        self.deck = self.deck[1:]

    def calc_probability_for_higher(self):
        hand = self.hand.copy()
        open_cards = self.open_cards.copy()

        if open_cards.shape[0] < 2:
            hand_val = get_combination_value(np.concatenate((hand,open_cards.reshape(-1,2))))
            if hand_val == 0:
                return (1 - np.max(hand[:,0])/13) * 0.5 + 0.5
            return cumulative_percentages[hand_val]

        #Make Dynamic
        trydeck = get_shuffled_deck()

        removes = []
        for i, c in enumerate(trydeck):
            if all(c == hand[0]) or all(c == hand[1]) or any([all(c == oc) for oc in open_cards]):
                removes.append(i)
                
        trydeck = np.delete(trydeck, removes, axis = 0)

        hands = []
        for ha in combinations(range(trydeck.shape[0]),5-open_cards.shape[0]):
            opp_cards = array([trydeck[h] for h in ha])
            open = np.concatenate((open_cards,opp_cards[:(3-open_cards.shape[0])]))
            tryhand = np.concatenate((open_cards, opp_cards))

            opp_val = get_hand_value(tryhand) 

            p = (np.concatenate((open,hand)))
            # print(open.shape,open_cards.shape,hand.shape)
            player_val = get_hand_value(p) 

            hands.append(opp_val>player_val)

        return sum(hands)/len(hands)   

    def play(self):
        self.start_game()
        self.draw_hand()
        self.draw_opponent_hand()

        val = 0

        print("Player hand:")
        print(self.opponent_hand)
        print("Open cards:")
        for i in range(3):
            maxpv = (1 - self.calc_probability_for_higher())*100
            if val > maxpv:
                print(f"Fold from bet {val} against a max of {maxpv}")
                break
            else:
                print(f"Bot stays")

                self.show_card()
            print("Open Cards:\n",self.open_cards)
            time.sleep(0.5)
            val += int(input("What do you want to bet? "))

        if self.open_cards.shape[0] == 3:
            pv = self.get_hand_value(self.hand)
            ov = self.get_hand_value(self.opponent_hand)

            if pv > ov:
                print(f"Bot won!")
            else:
                print(f"Bot lost :(")

            print("Bot hand was:\n",self.hand)
            print("Bot had a limit at",maxpv)
            print(f"Bot had {names[int((pv-pv%13)/13)]} with a high {pv%13}")
            print(f"Player had {names[int((ov-ov%13)/13)]} with a high {ov%13}")

# deck = get_shuffled_deck()

# for i in range(0,4):
#     hand, open = deck[:2], deck[2:(2+i)]
#     pro = calc_probability_for_higher(hand,open)
#     print(f"Hand: \n{hand[:2]}\nOpen: \n{open}\nProb: {pro*100:.3}\n")


game = Game()

game.play()

Player hand:
[[11  2]
 [ 9  3]]
Open cards:
Bot stays
Open Cards:
 [[11  1]]


ValueError: invalid literal for int() with base 10: ''

In [ ]:
def print_distribution(N = 3*10000):
    vals = np.empty(N*8)
    for i in range(N):
            
        deck = get_shuffled_deck()
        for k in range(8):
            val = get_hand_value(deck[k*5:(k+1)*5],array([[],[]]).T)
            vals[i*8+k] = val
            
    N = 3*10000
    vals = np.empty(N*8)
    for i in range(N):
            
        deck = get_shuffled_deck()
        for k in range(8):
            val = get_hand_value(deck[k*5:(k+1)*5],array([[],[]]).T)
            vals[i*8+k] = val

    t = np.unique(vals,return_counts = True)

    for a,b in zip(*t):
        print(a,b/sum(t[1])*100)

# ML

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

#### GAN-idé

1)

5 kort med farve, værdi og aktivering

Hvad der er blevet budt sidste hånd

Gæt om modstanderhånden er bedre?

2)

5 kort med farve, værdi og aktivering

Gæt hvad der skal bydes

In [182]:
N_in = 5*2 + 1
N_out = 1
SIZE = 16
batch_size = 16
epochs = 2*1000

In [183]:
guesser = keras.Sequential(
    [
        layers.InputLayer(input_shape=(batch_size,N_in)),
        layers.Dense(SIZE),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(SIZE),
        layers.LeakyReLU(alpha=0.2),
        # layers.GlobalMaxPooling2D(),
        layers.Dense(1,activation="sigmoid"),
    ],name = 'Guesser'
)

better = keras.Sequential(
    [
        layers.InputLayer(input_shape=(N_in - 1),name = "input"),
        layers.Dense(SIZE,name="2"),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(SIZE),
        layers.LeakyReLU(alpha=0.2),
        # layers.GlobalMaxPooling2D(),
        layers.Dense(N_out,activation="sigmoid")
    ],name = 'Better'
)


In [184]:
# loss_fn = keras.losses.BinaryCrossentropy()

def loss_fn(truths, guesses, bets):
    
    # For bet b
    #         | Wrong | Right
    # Guesser |   +b  |  -b
    # Better  |   -b  |  +b
    tg = guesses > 0
    
    hits = tg == truths

    bb = bets.copy()
    bb[hits] = -bb[hits]
    guess_loss = np.mean(bb)

    better_loss = np.mean(-bb)

    return (guess_loss, better_loss)

guess_optimizer = keras.optimizers.Adam(learning_rate=0.001)

bet_optimizer = keras.optimizers.Adam(learning_rate=0.001)

In [185]:
def generate_data(epochs):

    # Make vectorized

    all_hands = []
    for _ in range(epochs):
        guesser_hands = np.empty((batch_size,10),int)
        better_hands = np.empty((batch_size,10),int)
        truths = np.empty(batch_size,bool) # True if guesser is better
        for i in range(batch_size):
            deck = get_shuffled_deck()
            drawn = deck[:7]

            guesser_hand = drawn[:5]
            better_hand = drawn[2:]
            truth = get_hand_value(guesser_hand) > get_hand_value(better_hand)

            guesser_hands[i] = guesser_hand.flatten()
            better_hands[i] = better_hand.flatten()
            truths[i] = truth 

        all_hands.append((guesser_hands,better_hands,truths))
    return all_hands


In [186]:

all_hands = generate_data(epochs)

print("Beginning training")
for i in range(epochs):
    guesser_hands,better_hands,truths = all_hands[i]
    with tf.GradientTape() as tape:
        bet = better(better_hands)
        bet = tf.cast(tf.multiply(bet,10),"int32")

        gg = tf.concat((bet, guesser_hands),1)
        guess = tf.squeeze(guesser(gg))
        # print(truths.shape,tf.squeeze(guess).shape)
        loss = loss_fn(truths,guess,bet)

    if i % 50 == 0:
        print(keras.backend.eval(loss))
    grads = tape.gradient(loss, guesser.trainable_weights)
    guess_optimizer.apply_gradients(
        zip(grads, guesser.trainable_weights)
    )

Beginning training
0.90892833
1.0573957
0.683843
1.1882778
0.9612299
0.968652
0.72991025
0.92253697
0.75479174
1.0296391
0.99861145
0.85062206
0.86004615
0.915299
0.7240596
0.6671005
0.8912147
0.92677474
0.9676716
0.9641666
0.6623465
0.84084666


In [170]:
keras.backend.eval(loss)

1.2214209